In [7]:
import numpy as np
import pandas as pd
from scipy.stats import t, chi2

# --- 1. ЗАВАНТАЖЕННЯ ВИБІРКИ ---
df_data = pd.read_csv('chicken_population.csv')
data = df_data.iloc[:, 0].values

N = len(data)
alpha = 0.05 # Рівень значущості

print(f"Обсяг вибірки (n): {N}")
print(f"Мін: {data.min()}, Макс: {data.max()}")
print("-" * 60)

# --- 2. ОЦІНКА ПАРАМЕТРА (λ) ---
# Знаходимо середнє і потім λ = 1 / середнє
mean_val = np.mean(data)
lambda_hat = 1 / mean_val

print(f"Вибіркове середнє: {mean_val:.4f}")
print(f"Оцінка параметра λ (1 / середнє): {lambda_hat:.6f}")
print("-" * 60)

# --- 3. ФОРМУВАННЯ ІНТЕРВАЛІВ ---
# Розбиваємо діапазон від 0 до максимуму
# Робимо, наприклад, 10 кроків
num_bins = 10
edges = np.linspace(0, data.max(), num_bins + 1)

# Рахуємо реальні частоти в цих інтервалах
n_obs, _ = np.histogram(data, bins=edges)

# Формуємо список меж для зручності
intervals = []
for i in range(len(edges)-1):
    intervals.append((edges[i], edges[i+1]))

# --- 4. ОБЧИСЛЕННЯ ТЕОРЕТИЧНИХ ЧАСТОТ (n') ---
# Формула: n' = n * (e^(-λ*start) - e^(-λ*end))
n_theoretical_raw = []
for start, end in intervals:
    prob = np.exp(-lambda_hat * start) - np.exp(-lambda_hat * end)
    n_theoretical_raw.append(N * prob)

# Додаємо "хвіст" після останнього інтервалу до нескінченності,
# оскільки сума ймовірностей має бути 1.

# --- 5. ГРУПУВАННЯ МАЛИХ ЧАСТОТ ---
final_obs = []
final_exp = []
final_intervals = []

current_obs = 0
current_exp = 0
current_labels = []

# Йдемо по інтервалах і об'єднуємо, якщо n' < 5
for i in range(len(n_obs)):
    current_obs += n_obs[i]
    current_exp += n_theoretical_raw[i]

    lbl = f"{intervals[i][0]:.0f}-{intervals[i][1]:.0f}"
    current_labels.append(lbl)

    # Умова закриття групи
    if current_exp >= 5:
        final_obs.append(current_obs)
        final_exp.append(current_exp)

        start_val = current_labels[0].split('-')[0]
        end_val = current_labels[-1].split('-')[1]
        final_intervals.append(f"{start_val}-{end_val}")

        current_obs = 0
        current_exp = 0
        current_labels = []

# Обробка останнього залишку (якщо він залишився і < 5)
if current_exp > 0:
    if len(final_obs) > 0:
        final_obs[-1] += current_obs
        final_exp[-1] += current_exp
        # Оновлюємо назву
        prev_start = final_intervals[-1].split('-')[0]
        last_end = current_labels[-1].split('-')[1]
        final_intervals[-1] = f"{prev_start}-{last_end}"
    else:
        final_obs.append(current_obs)
        final_exp.append(current_exp)
        final_intervals.append("Всі дані")

# --- 6. ТАБЛИЦЯ І ХІ-КВАДРАТ ---
df_table = pd.DataFrame({
    'Інтервал': final_intervals,
    'ni (Емпіричні)': final_obs,
    "n'i (Теоретичні)": np.round(final_exp, 4)
})

# Розрахунок доданку хі-квадрат
df_table["(ni - n'i)^2 / n'i"] = (df_table['ni (Емпіричні)'] - df_table["n'i (Теоретичні)"])**2 / df_table["n'i (Теоретичні)"]

print("Розрахункова таблиця:")
print(df_table)
print("-" * 60)

chi_sq_calc = df_table["(ni - n'i)^2 / n'i"].sum()

# Ступені вільності: m = k - 1 - s
# s = 1 (оцінили один параметр λ)
k = len(final_obs)
s = 1
dof = k - 1 - s

print(f"Спостережуване значення χ²: {chi_sq_calc:.4f}")

if dof <= 0:
    print("Неможливо коректно провести тест: занадто мало груп.")
else:
    chi_sq_crit = chi2.ppf(1 - alpha, dof)
    print(f"Ступені вільності (k={k} - 1 - s={s}): {dof}")
    print(f"Критичне значення χ² (при α={alpha}): {chi_sq_crit:.4f}")
    print("-" * 60)

    if chi_sq_calc < chi_sq_crit:
        print("ВИСНОВОК: Гіпотеза про показниковий розподіл ПРИЙМАЄТЬСЯ.")
    else:
        print("ВИСНОВОК: Гіпотеза про показниковий розподіл ВІДХИЛЯЄТЬСЯ.")

Обсяг вибірки (n): 99
Мін: 187, Макс: 574
------------------------------------------------------------
Вибіркове середнє: 296.5859
Оцінка параметра λ (1 / середнє): 0.003372
------------------------------------------------------------
Розрахункова таблиця:
  Інтервал  ni (Емпіричні)  n'i (Теоретичні)  (ni - n'i)^2 / n'i
0     0-57               0           17.4200           17.420000
1   57-115               0           14.3548           14.354800
2  115-172               0           11.8289           11.828900
3  172-230              10            9.7475            0.006541
4  230-287              35            8.0323           90.541544
5  287-344              38            6.6190          148.778843
6  344-402              12            5.4543            7.855488
7  402-574               4           11.2503            4.672484
------------------------------------------------------------
Спостережуване значення χ²: 295.4586
Ступені вільності (k=8 - 1 - s=1): 6
Критичне значення χ² (п